In [3]:
from bs4 import BeautifulSoup as bs
import urllib3
import requests
import urllib.request
import pickle
import os

In [4]:
!rucio whoami

account    : egazzarr
status     : ACTIVE
email      : elena.gazzarrini@cern.ch
deleted_at : None
updated_at : 2021-10-07T09:30:20
account_type : SERVICE
suspended_at : None
created_at : 2021-10-07T09:30:20


In [5]:
link_sc='https://heasarc.gsfc.nasa.gov/FTP/fermi/data/lat/weekly/spacecraft/'
link_p8='https://heasarc.gsfc.nasa.gov/FTP/fermi/data/lat/weekly/p8r2/photon/'
link_p='https://heasarc.gsfc.nasa.gov/FTP/fermi/data/lat/weekly/photon/'

#FILES WRITTEN to a file to check if they already exist
out_sc_f= './fermilat_sc.txt'
out_p8_f='./fermilat_p8.txt'
out_p_f='./fermilat_p.txt'

# directory where new files are downloaded, uploaded and deleted immediately after to not occupy too much local memory
out_p8='./'
out_p='./'
out_sc= './'

#rucio scope and rse names 
scope='FERMILAT_LAPP_SP'
rse='CNAF-STORM'

#rucio dataset names  
dataset_sc='weekly.spacecraft'
dataset_p='weekly.photon'
dataset_p8='weekly.photonp8r2'

In [6]:
print('Spacecraft files sync')

page = requests.get(link_sc, allow_redirects=True)
soup=bs(page.content)

"""Uncomment if you want to add a new dataset and see what is already in the datasets"""
# !rucio add-dataset $scope:$dataset_sc
# print('Ignore the error message from Rucio: "Data Identifier Already Exists.", as this means the Rucio DID (scope:dataset) has already been created. This command will only be executed if dataset doesnt exist already.')
# !rucio list-files $scope:$dataset_sc

try:
    f = open(out_sc_f, 'r')
    name_list_sc=[line.rstrip() for line in f]
    f.close()
    print('The .txt file already exists. Will append new web scraped files. The list contains', len(name_list_sc), 'elements')
except:
    f=open(out_sc_f, 'w')
    name_list_sc=[]
    f.close()
    print('The .txt file does not exist yet, creating it. The list looks like: ', name_list_sc)
    
# !rucio list-files $scope:$dataset_sc
    
for tag in soup.find_all('a')[5:]:
    try:
        if not tag:
            continue
        name = tag.text
        if name not in name_list_sc and 'fits' in name:
            #download the file in your local direcotry
            urllib.request.urlretrieve(link_sc+name, str(out_sc+name))
            print('downloaded ', name)
            try:  
                file=out_sc+name
                !rucio upload $file --scope $scope --rse $rse --register-after-upload --protocol davs 
            except:
                print('could not upload file')
                break 
            else:
                print('uploaded')
                try:
                    !rucio attach $scope:$dataset_sc $scope:$name
                except:
                    print('could not attach file')
                    break 
                else:
                    print('attached, ready to remove file')
                    os.remove(file)
                    print('removed')  
                    #append to list if it doesn't already exist
                    name_list_sc.append(name)
                    #appends to the .txt file
                    with open (out_sc_f, 'a') as f:
                        f.write(name+'\n')
    except:
        pass
    else:
        pass
    
print('no new files to upload for spacecraft')

Spacecraft files sync
The .txt file already exists. Will append new web scraped files. The list contains 719 elements
no new files to upload for spacecraft


In [ ]:
print('photon p8r2 files sync')

page = requests.get(link_p8, allow_redirects=True)
soup=bs(page.content)


"""Uncomment if you want to add a new dataset and see what is already in the datasets"""
# !rucio add-dataset $scope:$dataset_p8
# print('Ignore the error message from Rucio: "Data Identifier Already Exists.", as this means the Rucio DID (scope:dataset) has already been created. This command will only be executed if dataset doesnt exist already.')
# !rucio list-files $scope:$dataset_p8

try:
    f = open(out_p8_f, 'r')
    name_list_p8=[line.rstrip() for line in f]
    f.close()
    print('The .txt file already exists. Will append new web scraped files.  The list contains', len(name_list_p8), 'elements')
except:
    f=open(out_p8_f, 'w')
    name_list_p8=[]
    f.close()
    print('The .txt file does not exist yet, creating it. The list looks like: ', name_list_p8)

        
for tag in soup.find_all('a')[5:]:
    try:
        if not tag:
            continue
        name = tag.text
        if name not in name_list_p8 and 'fits' in name:
            #download the file in your local direcotry
            urllib.request.urlretrieve(link_p8+name, str(out_p8+name))
            print('downloaded ', name)
            try:  
                file=out_p8+name 
                !rucio upload $file --scope $scope --rse $rse --register-after-upload --protocol davs 
            except:
                print('could not upload file')
                break 
            else:
                print('uploaded')
                try:
                    !rucio attach $scope:$dataset_p8 $scope:$name
                except:
                    print('could not attach file')
                    break 
                else:
                    print('attached, ready to remove file')
                    os.remove(file)
                    print('removed')  
                    #append to list if it doesn't already exist
                    name_list_p8.append(name)
                    #appends to the .txt file
                    with open (out_p8_f, 'a') as f:
                        f.write(name+'\n')
    except:
        pass
    else:
        pass
print('no new files to upload for photon p8r2')

In [ ]:
print('photon p8r3 files sync')

page = requests.get(link_p, allow_redirects=True)
soup=bs(page.content)


"""Uncomment if you want to add a new dataset and see what is already in the datasets"""
# !rucio add-dataset $scope:$dataset_p
# print('Ignore the error message from Rucio: "Data Identifier Already Exists.", as this means the Rucio DID (scope:dataset) has already been created. This command will only be executed if dataset doesnt exist already.')
# !rucio list-files $scope:$dataset_p

try:
    f = open(out_p_f, 'r')
    name_list_p=[line.rstrip() for line in f]
    f.close()
    print('The .txt file already exists. Will append new web scraped files.  The list contains', len(name_list_p), 'elements')
except:
    f=open(out_p_f, 'w')
    name_list_p=[]
    f.close()
    print('The .txt file does not exist yet, creating it. The list looks like: ', name_list_p)
    
for tag in soup.find_all('a')[5:]:
    try:
        if not tag:
            continue
        name = tag.text
        if name not in name_list_p and 'fits' in name:
            #download the file in your local direcotry
            urllib.request.urlretrieve(link_p +name, str(out_p+name))
            print('downloaded ', name)
            try:  
                file=out_p +name
                !rucio upload $file --scope $scope --rse $rse --register-after-upload --protocol davs
            except:
                print('could not upload file')
                break 
            else:
                print('uploaded')
                try:
                    !rucio attach $scope:$dataset_p $scope:$name
                except:
                    print('could not attach file')
                    break 
                else:
                    print('attached, ready to remove file')
                    os.remove(file)
                    print('removed')  
                    #append to list if it doesn't already exist
                    name_list_p.append(name)
                    #appends to the .txt file
                    with open (out_p_f, 'a') as f:
                        f.write(name+'\n')
    except:
        pass
    else:
        pass
print('no new files to upload for photon')